In [ ]:
# making processed M5 data 

In [2]:
import pandas as pd

In [3]:
train = pd.read_csv(('sales_train_validation.csv'),  encoding='utf-8',engine='c')

# get days which have time series values
day_cols = [col for col in train.columns if col.startswith('d_')]
idx = [int(col.split('d_')[1]) for col in day_cols]

In [4]:
# Ensures uniqueness of category, dept, and item across hierarchie
train['cat_id']  = (train['store_id'] + '_' + train['cat_id'])
train['dept_id'] = (train['store_id'] + '_' + train['dept_id'])
train['item_id'] = (train['store_id'] + '_' + train['item_id'])

In [6]:
# make date time
calendar = pd.read_csv('calendar.csv')
train_date_id = pd.to_datetime(calendar.date[:1913])

In [9]:
def transpose(column, index, day_col):
    """
    Turn the row oriented time series into a column oriented one 
    """
    ts = []
    new_cols = train[column].unique()
    
    for value in new_cols:
        value_ts = train[train[column] == value]
        vertical = value_ts[day_col].sum().T
        vertical.index = index
        ts.append(vertical)
    return pd.DataFrame({k: v for k, v in zip(new_cols, ts)})

In [10]:
# it may takes some minutes
state_ts = transpose('state_id', train_date_id, day_cols)
store_ts = transpose('store_id', train_date_id, day_cols)
cat_ts = transpose('cat_id', train_date_id, day_cols)
dept_ts = transpose('dept_id', train_date_id, day_cols)
item_ts = transpose('item_id', train_date_id, day_cols)

In [12]:
df = pd.concat([state_ts, store_ts, cat_ts,dept_ts, item_ts], 1)

# Total column is the root node, i.e., the sum
# of all demand across all stores we have data for
df['total'] = df['CA'] + df['TX'] + df['WI']

df.head()

,CA,TX,WI,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,...,WI_3_FOODS_3_819,WI_3_FOODS_3_820,WI_3_FOODS_3_821,WI_3_FOODS_3_822,WI_3_FOODS_3_823,WI_3_FOODS_3_824,WI_3_FOODS_3_825,WI_3_FOODS_3_826,WI_3_FOODS_3_827,total
date,,,,,,,,,,,,,,,,,,,,,
2011-01-29,14195,9438,8998,4337,3494,4739,1625,2556,3852,3030,...,14,1,0,4,0,0,0,0,0,32631
2011-01-30,13805,9630,8314,4155,3046,4827,1777,2687,3937,3006,...,11,1,0,4,0,0,6,0,0,31749
2011-01-31,10108,6778,6897,2816,2121,3785,1386,1822,2731,2225,...,5,1,0,2,2,0,0,0,0,23783
2011-02-01,11047,7381,6984,3051,2324,4232,1440,2258,2954,2169,...,6,1,0,5,2,0,2,0,0,25412
2011-02-02,9925,5912,3309,2630,1942,3817,1536,1694,2492,1726,...,5,1,0,2,0,0,2,0,0,19146


In [16]:
# df.to_csv(('processed_M5_hierarchyN.csv'))